In [3]:
from sympy import *
from sympy import Function, Symbol

The SIR model is given by

\begin{equation} \begin{split} S'&= -kSI\\ I'&= kSI-qI\\R'&= qI
\end{split}\end{equation}
subject to $S+I+R=1$,
where $S$ represents the proportion of the total population that is susceptible to the disease, $I$ represents the proportion of the total population that is infected, and $R$ represents the proportion of the total population that is recovered from the disease.

In [4]:
# initialize ODE dependent variables
S, I, R = symbols('S,I,R')

# initialize parmeters
k, q = symbols('k,q')

sys_vars = [S,I,R] # vector of ODE variables
params = [k,q] # vector of parameters

Id,Rd = symbols('Id,Rd') # data for the variables

# functional we wish to minimize
J_integrand = (I-Id)**2+(R-Rd)**2

# Right hand side of the ODE (must be first order IVP)
RHS_ODE = [-k*S*I,k*S*I-q*I,q*I]


In [5]:
# create adjoint variables
advars = []
for j in range(len(sys_vars)):
  advars.append(sympify("P_{}".format(str(sys_vars[j]))))
print(advars)

[P_S, P_I, P_R]


In [7]:
# Find the RHS of the adjoint equations.

# WARNING: This assumes that J looks like $\int_0^T (I-I_{observations})^2dt$ and not like $\int_0^T k*(I-I_{observations})^2dt$.

# WARNING: Make sure the sign is right for these based on how you set up your augmented functional.

d_sys = []
for j in range(len(sys_vars)):
  derivative = diff(J_integrand, sys_vars[j])
  for i, rhs in enumerate(RHS_ODE):
    derivative -= diff(rhs, sys_vars[j])*advars[i]
  d_sys.append(derivative)

d_sys

[-I*P_I*k + I*P_S*k, 2*I - 2*Id - P_I*(S*k - q) - P_R*q + P_S*S*k, 2*R - 2*Rd]

In [9]:
d_param = []
for j in range(len(params)):
  derivative = diff(J_integrand, params[j])
  for i, rhs in enumerate(RHS_ODE):
    derivative -= diff(rhs, params[j])*advars[i]
  d_param.append(derivative)

d_param

[-I*P_I*S + I*P_S*S, I*P_I - I*P_R]

In [10]:
d_param[0]

-I*P_I*S + I*P_S*S